# 第八课 并行训练算法

经过前面七课的学习，我们已经可以基于强化学习训练自己的AI选手了。为了加快训练速度，可以采用并行训练的方法，本节课就来学习如何进行并行训练。并行训练需要同时运行多个数字冰壶服务器实例，受限于平台算力支持以及网络连接稳定性，在训练时需要使用单机版数字冰壶服务器。

## 8.1 单机版数字冰壶服务器

这门线上数字冰壶课程中的所有内容都需要结合数字冰壶服务器才能完成编程实践，方便同学们快速上手从零开始学习编写数字冰壶AI。大家在掌握了核心理论与算法之后，就可以<a href="https://aichallenge.embedded-ai.org/api/download-file?id=6688bd33a607189b339e956b">>>下载单机版数字冰壶服务器<<</a>，在本地调试自己的数字冰壶AI程序了。

下载好的单机版数字冰壶客户端是一个压缩包文件，解压缩后的目录中可以看到如下所示的内容：

├─conf<br>
│　└─config.ini：配置文件<br>
├─CurlingAI<br>
│　├─player1<br>
│　│　└─CurlingAI.exe：测试用AI选手1<br>
│　└─player2：<br>
│　 　└─CurlingAI.exe：测试用AI选手2<br>
├─resource<br>
│  └─robot.py：数字冰壶AI选手python范例代码（投掷调试模式下的陪练AI）<br>
├─web<br>
│  └─index.html：单机版数字冰壶服务器WEB页面<br>
├─curling.log：日志文件<br>
├─curling_server.exe：单机版数字冰壶服务器主程序<br>
└─CurlingUI：调用浏览器加载单机版数字冰壶服务器WEB页面的快捷方式<br>

打开配置文件conf/config.ini，可以看到如下所示的内容：

编辑此配置文件可以修改单机版数字冰壶服务器的WEB端口（HttpPort）和TCP端口（TcpPort），同时运行多个数字冰壶服务器的时候需要修改这两个端口。

运行单机版数字冰壶服务器主程序curling_server.exe，可以看到如图所示的界面。

<center><img src="img/SingleServer1.png" width=800 border=1></center>

从界面中也可以看到当前运行的单机版数字冰壶服务器的WEB端口是9007、TCP端口是7788。

运行快捷方式CurlingUI，或者手动调用浏览器访问地址localhost:9007（配置文件中的HttpPort），即可看到熟悉的数字冰壶服务器界面，如下图所示。

<center><img src="img/SingleServer2.png" width=800></center>

从图中可以看到单机版数字冰壶服务器界面右下角没有连接信息，这是因为单机版数字冰壶服务器不验证接入AI的密钥，提供任意密钥都可以完成连接。

在单机版数字冰壶服务器界面上点击【四局制】，分别运行测试用AI选手CurlingAI/player1/CurlingAI.exe和CurlingAI/player2/CurlingAI.exe，运行界面如下图所示。

<center><img src="img/SingleServer3.png" width=800></center>

即会在单机版数字冰壶服务器界面上看到两个测试用AI选手已连接。依次点击【准备】和【开始对局】，看到单机版数字冰壶服务器界面中两个AI选手开始对战，即表明单机版数字冰壶服务器一切功能正常。

<b>确保本地PC机上已经自行安装好python程序的运行环境</b>，将课程平台上的编写好的数字冰壶AI选手的python源码拷贝/下载到本地，<b>修改源码中的服务器主机（本地访问都是127.0.0.1）和端口（配置文件中的TcpPort）</b>，运行python程序即可在本地对数字冰壶AI选手进行调试。

## 8.2 并行DQN模型的训练与部署

深度Q网络（DQN）是强化学习中的一个里程碑算法，它结合了Q学习与深度神经网络。并行DQN是对标准DQN的扩展，旨在通过并行化来提高学习效率和性能。

### 8.2.1 并行DQN模型训练的专有组件

并行DQN模型的训练过程主要包含以下组件：

1. 全局DQN模型

全局DQN模型是所有Agent共享的核心。它包含两个网络：一个用于初始化（init），另一个用于后续操作（dote）。这种设计允许系统适应不同阶段的任务特性。

2. 多个并行运行的Agent

每个Agent是一个独立的实体，它们并行运行，与环境交互，并贡献到全局模型的学习过程。这种并行架构借鉴了A3C算法的思想。

3. 共享内存（经验回放缓冲区）

共享内存实现了经验回放机制，这是DQN算法的关键组成部分。在并行设置中，所有Agent共享这个内存，增加了经验的多样性。

4. 参数同步机制

参数同步确保所有Agent的本地模型与全局模型保持一致。这是分布式学习系统的关键特性。

学习过程：Agents与环境交互，将经验存储在共享内存中。这个过程类似于标准DQN，但在并行环境中进行。包括从共享内存采样，计算损失，反向传播，更新全局模型。

### 8.2.2 并行DQN模型训练的范例代码

并行DQN模型训练的示例代码如下所示。使用时需要根据agent的设置数量（参考本地PC机的CPU内核数量）打开同等数量的数字冰壶单机版副本，需要注意每个数字冰壶单机版需要进入conf文件夹修改config.ini中的HttpPort以及TcpPort，每个线程连接不同的Port，从而实现多组agent同时对局。

> 下方范例代码需要拷贝到本地PC上配合单机版数字冰壶服务器运行，请【不要】尝试运行下方代码单元。

### 8.2.3 并行DQN模型训练的实操流程

#### >> 启动多个数字冰壶比赛服务器

上面给出的的范例代码中，同时启动了两个并发的训练进程，端口分别是默认的7788和新增的7789。为了支持这两个并发的训练进程，需要在本地PC上同时开启两个单机版数字冰壶服务器。

将“数字冰壶单机版”目录整体在本地PC硬盘中复制粘贴，得到“数字冰壶单机版 - 副本”目录，编辑副本目录下配置文件conf/config.ini，更改HTTP端口（HttpPort）和TCP端口（TcpPort）的值，范例如下所示：

运行副本目录下单机版数字冰壶服务器主程序curling_server.exe，手动调用浏览器访问地址localhost:9009加载副本目录下的单机版数字冰壶服务器主页。

在两个单机版数字冰壶服务器的界面中进行操作，都进入【无限局制】模式。

#### >> 运行多进程训练DQN模型的AI选手

在当前课程页面左侧目录树区域中可以看到一个名为AIRobot.py的Python脚本，将该脚本下载到本地硬盘上。在AIRobot.py的保存目录下新建python文件（假设文件名是DQNRobot.py），将上方代码单元中的代码拷贝粘贴到DQNRobot.py中。

准备工作：

1. 确保本地PC机上已经自行安装好python程序的运行环境；
2. 将在当前课程页面左侧目录树区域中的AIRobot.py脚本下载到本地硬盘上；
3. 在AIRobot.py的保存目录下新建log子目录和model子目录，注意大小写；
4. 在AIRobot.py的保存目录下新建DQNRobot.py脚本文件，将上方代码单元中的代码拷贝粘贴到新建文件中。

运行DQNRobot.py，该脚本会先后启动两个并发的训练进程，分别连接到当前正在运行的两个单机版数字冰壶服务器。如下图所示。

<center><img src="img/ParallelDQN1.png" width=800></center>

#### >> 运行基础AI选手

运行“数字冰壶单机版”目录下的测试用AI选手CurlingAI/player1/CurlingAI.exe做为7788端口数字冰壶服务器上的基础AI陪练。

> 也可运行python脚本AIRobt.py做为7788端口数字冰壶服务器上的基础AI陪练。

测试用AI选手CurlingAI.exe默认都是连接本机的7788端口，我们需要带参数调用该程序以连接7789端口数字冰壶服务器。在资源管理器中进入“数字冰壶单机版 - 副本”目录下的CurlingAI/player1/目录中，右键点击CurlingAI.exe选择创建快捷方式，再右键点击新建的快捷方式编辑其属性，修改目标的值为`"D:\数字冰壶单机版 - 副本\CurlingAI\player1\CurlingAI.exe" -p 7789`。双击运行该快捷方式，将“数字冰壶单机版 - 副本”目录下的测试用AI选手CurlingAI/player1/CurlingAI.exe做为7789端口数字冰壶服务器上的基础AI陪练。

> 也可带参数运行python脚本AIRobt.py做为7789端口数字冰壶服务器上的基础AI陪练。<br>`python AIRobot.py -p 7789`

#### >> 在无限对战中开始训练

分别在浏览器的两个单机版数字冰壶服务器界面上确认对战双方AI选手已连接，依次点击【准备】和【开始对局】，即可开始并行DQN模型的训练。

> 1. 为提高训练效率，训练全程都要保证两个单机版数字冰壶服务器的WEB页面在前台运行（窗口模式且露出部分窗口在屏幕上即可）。<br>
2. 因为两个进程都是50轮对局保存一次模型，最好一个进程晚开始几分钟，将保存时间错开。


<center><img src="img/ParallelDQN2.png" width=800></center>

在模型训练的过程中，随时可以通过在数字冰壶服务器界面中点击【返回主菜单】停止训练。

## 8.3 并行PPO模型的训练与部署

运用相同的思路也可以得到并行PPO的代码：

1. 使用SharedAdam优化器来支持参数的共享内存访问。
2. 创建全局共享的Actor和Critic网络，以及相应的优化器。
3. 每个Agent有自己的本地Actor和Critic网络，定期与全局网络同步。
4. 使用共享内存队列(self.shared_memory)来实现经验回放缓冲区的共享。
5. 在训练时，从共享内存中获取经验数据，更新本地模型，然后将梯度应用到全局模型。
6. 定期将全局模型的参数同步回本地模型。

这种实现方式允许多个Agent并行地与环境交互和学习，同时共享和更新一个全局模型。这可以加速学习过程，并可能提高模型的泛化能力。

范例代码如下所示，具体实操流程和并行训练DQN模型的实操流程类似，不再赘述。

> 下方范例代码需要拷贝到本地PC上配合单机版数字冰壶服务器运行，请【不要】尝试运行下方代码单元。

## 8.4 并行模型训练方法的范例代码优化

前面两个并行模型训练方法的示例代码中都存在参数覆盖问题，下面提供几种解决方法，同学可以自行修改代码进行实验：

1. 使用锁机制：这是最直接的方法，但可能会降低并行效率。

2. 梯度累积：在本地累积梯度，然后定期更新全局模型。

3. 参数平均：定期收集所有本地模型的参数，计算平均值作为新的全局模型。

4. 异步参数服务器：实现一个中央参数服务器来管理更新。

5. 使用分布式数据并行：利用PyTorch的DistributedDataParallel来自动处理梯度同步和更新。

6. 弹性平均SGD：允许本地模型在一定程度上偏离全局模型

## 小结

本课讲解了并行DQN模型训练和并行PPO模型训练的原理，并分别给出了范例代码，进一步还讲解了如何结合单机版数字冰壶服务器在本地PC机上实现并行DQN模型训练和并行PPO模型训练。

强化学习模型都需要上万局才能看到训练效果，并行模型训练方式可以在本地PC上启动数个进程（进程数量取决于本地PC机的CPU核数和内存容量），可以大大提高训练效率。

另外需要注意的是，在本教程里提供的各种强化学习算法的范例代码，都仅演示了基本的功能，还有大量需要优化的细节，需要大家自行研究。